In [38]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='1wMTDB8Qanj42QYh_DAQCd2nC69KTk2jsl9DyiKs8UN8',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.au-syd.cloud-object-storage.appdomain.cloud')

bucket = 'pmgsy-donotdelete-pr-dbgv1zpqnn5p03'
object_key = 'PMGSY_DATASET.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_4 = pd.read_csv(body)
df_4.head(10)


,STATE_NAME,DISTRICT_NAME,PMGSY_SCHEME,NO_OF_ROAD_WORK_SANCTIONED,LENGTH_OF_ROAD_WORK_SANCTIONED,NO_OF_BRIDGES_SANCTIONED,COST_OF_WORKS_SANCTIONED,NO_OF_ROAD_WORKS_COMPLETED,LENGTH_OF_ROAD_WORK_COMPLETED,NO_OF_BRIDGES_COMPLETED,EXPENDITURE_OCCURED,NO_OF_ROAD_WORKS_BALANCE,LENGTH_OF_ROAD_WORK_BALANCE,NO_OF_BRIDGES_BALANCE,Unnamed: 14
0,Andaman And Nicobar,Nicobar,PMGSY-I,3,2.969,0,1.38570,3,2.969,0,1.1082,0,0.000,0,NaN
1,Andaman And Nicobar,North and Middle Andaman,PMGSY-I,32,60.169,0,24.69080,32,59.190,0,18.5754,0,0.000,0,NaN
2,Andaman And Nicobar,North and Middle Andaman,PMGSY-II,24,54.670,0,27.76460,0,0.000,0,0.0000,24,54.670,0,NaN
3,Andaman And Nicobar,South Andaman,PMGSY-I,32,40.146,0,18.53750,31,39.287,0,11.6894,1,0.541,0,NaN
4,Andaman And Nicobar,South Andaman,PMGSY-II,24,41.992,0,23.42439,7,17.747,0,8.6343,17,24.033,0,NaN
5,Andhra Pradesh,Anantapur,PMGSY-I,619,2169.505,35,526.12770,619,2126.386,34,517.9120,0,0.000,1,NaN
6,Andhra Pradesh,Anantapur,PMGSY-II,14,125.700,0,56.07110,14,124.619,0,49.0126,0,0.000,0,NaN
7,Andhra Pradesh,Anantapur,PMGSY-III,27,267.158,8,180.52038,24,234.902,0,111.5182,3,23.200,8,NaN
8,Andhra Pradesh,Chittoor,PMGSY-I,283,889.681,6,188.35580,283,878.711,6,185.1493,0,0.000,0,NaN
9,Andhra Pradesh,Chittoor,PMGSY-II,18,126.045,0,57.96680,18,121.403,0,46.1360,0,0.000,0,NaN


In [42]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_4['PMGSY_SCHEME'] = le.fit_transform(df_4['PMGSY_SCHEME'])

# Optional: See the mapping
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Label mapping:", label_mapping)


Label mapping: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4}


In [43]:
X = df_4.drop(columns=['PMGSY_SCHEME', 'STATE_NAME', 'DISTRICT_NAME'])
y = df_4['PMGSY_SCHEME']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [44]:
print(df_4.isnull().sum())


STATE_NAME                           0
DISTRICT_NAME                        0
PMGSY_SCHEME                         0
NO_OF_ROAD_WORK_SANCTIONED           0
LENGTH_OF_ROAD_WORK_SANCTIONED       0
NO_OF_BRIDGES_SANCTIONED             0
COST_OF_WORKS_SANCTIONED            52
NO_OF_ROAD_WORKS_COMPLETED           0
LENGTH_OF_ROAD_WORK_COMPLETED        0
NO_OF_BRIDGES_COMPLETED              0
EXPENDITURE_OCCURED                  0
NO_OF_ROAD_WORKS_BALANCE             0
LENGTH_OF_ROAD_WORK_BALANCE          0
NO_OF_BRIDGES_BALANCE                0
Unnamed: 14                       2189
dtype: int64


In [45]:
df_4.drop(columns=['Unnamed: 14'], inplace=True)


In [46]:
df_4['COST_OF_WORKS_SANCTIONED'].fillna(df_4['COST_OF_WORKS_SANCTIONED'].mean(), inplace=True)


In [47]:
print(df_4.isnull().sum())


STATE_NAME                        0
DISTRICT_NAME                     0
PMGSY_SCHEME                      0
NO_OF_ROAD_WORK_SANCTIONED        0
LENGTH_OF_ROAD_WORK_SANCTIONED    0
NO_OF_BRIDGES_SANCTIONED          0
COST_OF_WORKS_SANCTIONED          0
NO_OF_ROAD_WORKS_COMPLETED        0
LENGTH_OF_ROAD_WORK_COMPLETED     0
NO_OF_BRIDGES_COMPLETED           0
EXPENDITURE_OCCURED               0
NO_OF_ROAD_WORKS_BALANCE          0
LENGTH_OF_ROAD_WORK_BALANCE       0
NO_OF_BRIDGES_BALANCE             0
dtype: int64


In [48]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_4['STATE_NAME'] = le.fit_transform(df_4['STATE_NAME'])
df_4['DISTRICT_NAME'] = le.fit_transform(df_4['DISTRICT_NAME'])
df_4['PMGSY_SCHEME'] = le.fit_transform(df_4['PMGSY_SCHEME'])  # Target


In [49]:
X = df_4.drop(columns=['PMGSY_SCHEME'])  # Features
y = df_4['PMGSY_SCHEME']                 # Target


In [50]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [51]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8926940639269406
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       0.96      0.92      0.94       143
           2       0.86      0.94      0.90       128
           3       0.86      0.89      0.87       150
           4       0.50      0.09      0.15        11

    accuracy                           0.89       438
   macro avg       0.84      0.77      0.77       438
weighted avg       0.89      0.89      0.89       438

